### Install required dependencies

In [ ]:
!pip install numpy

### Load dataset

In [4]:
import numpy as np

# I took the function from tutorials
def prepare_data(filename):
    patterns = []
    with open(filename) as f:
        count, width, height = [int(x) for x in f.readline().split()] # header
        dim = width*height

        for _ in range(count):
            f.readline() # skip empty line
            x = np.empty((height, width))
            for r in range(height):
                x[r,:] = np.array(list(f.readline().strip())) == '#'
            patterns.append(2*x.flatten()-1) # flatten to 1D vector, rescale {0,1} -> {-1,+1}
    
    return patterns, dim

patterns, dim = prepare_data('data.in')

for p in patterns:
    print(p)

[-1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1.  1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1. -1.  1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1.]
[-1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1.  1.  1.
  1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1. -1.  1. -1.]
[-1. -1.  1.  1.  1. -1. -1. -1.  1. -1. -1. -1.  1. -1. -1.  1. -1. -1.
 -1.  1. -1. -1.  1. -1. -1. -1.  1. -1. -1. -1.  1.  1.  1. -1. -1.]
[-1.  1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1.
 -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1. -1.]


### Train hopfield